In [45]:
import shelve
import time
from datetime import datetime

_cache = shelve.open("cache")
TIMEOUT = 86400 * 30
def is_expired(dt, limit_seconds = 86400*7):
    return (datetime.now() - datetime.fromisoformat(dt)).seconds > limit_seconds

def clear_cache():
    _cache.clear()
    _cache.sync()

def cache(fun):
    def to_url(*args, **kwargs):
        url = kwargs['url'] if 'url' in kwargs else args[0]
        params_dict = kwargs['params'] if 'params' in kwargs else args[1] if len(args) > 1 else dict()
        params = '&'.join([str(k) + '=' + str(v) for (k,v) in params_dict.items()])
        return url + "?" + params
        
    def wrapper(*args, **kwargs):
        url = to_url(*args, **kwargs)
        print("url: " + url)
        if (url not in _cache):
            print("Not cached " + url)
            result = fun(*args, **kwargs)
            _cache[url] = {
                'url': url,
                'timestamp': datetime.now().isoformat(),
                'result': result
            }
            _cache.sync()
            return result
        
        result = _cache[url]
        if(not is_expired(result['timestamp'], TIMEOUT)):
            print("Cached " + url)
            return result['result']
        
        print("Expired " + url)
        result = fun(*args, **kwargs)
        _cache[url] = {
            'url': url,
            'timestamp': datetime.now().isoformat(),
            'result': result
        }
        _cache.sync()
        return result
        
    return wrapper

In [116]:
# Create schema
import sqlite3
conn = sqlite3.connect('moex.db')
try:
    c = conn.cursor()
    c.execute(
        '''CREATE TABLE IF NOT EXISTS engines (
            id int, 
            name text, 
            title text
        )''')
    c.execute(
        '''CREATE TABLE IF NOT EXISTS markets (
            id int, 
            trade_engine_id int,
            trade_engine_name text, 
            trade_engine_title text,
            market_name text,
            market_title text,
            market_id int,
            marketplace text
        )''')

    c.execute(
        '''CREATE TABLE IF NOT EXISTS boards (
            id int,
            board_group_id int,
            engine_id int,
            market_id int,
            boardid int,
            board_title text,
            is_traded int,
            has_candles int,
            is_primary int
        )''')

    c.execute(
        '''CREATE TABLE IF NOT EXISTS boardgroups (
            id int,
            trade_engine_id int,
            trade_engine_name text,
            trade_engine_title text,
            market_id int,
            market_name text,
            name text,
            title text,
            is_default int,
            board_group_id int,
            is_traded int
        )''')
    c.execute(
        '''CREATE TABLE IF NOT EXISTS durations (
            interval int,
            duration int,
            days int,
            title text,
            hint text
        )''')
    c.execute(
        '''CREATE TABLE IF NOT EXISTS securitytypes (
            id int,
            trade_engine_id int,
            trade_engine_name text,
            trade_engine_title text,
            security_type_name text,
            security_type_title text
        )''')
    c.execute(
        '''CREATE TABLE IF NOT EXISTS securitygroups (
            id int,
            name text,
            title text,
            is_hidden int
        )''')
    c.execute(
        '''CREATE TABLE IF NOT EXISTS securitycollections (
            id int,
            name text,
            title text,
            security_group_id int
        )''')
finally:
    conn.close()

In [117]:
def fill_engines(engines):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM engines')
        conn.commit()
        for e in engines:
            conn.execute('insert into engines values (?,?,?)',(
                e['id'],
                e['name'],
                e['title']
            ))
        conn.commit()
    finally:
        conn.close()

In [118]:
def fill_markets(markets):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM markets')
        conn.commit()
        for e in markets:                        
            conn.execute('insert into markets values (?,?,?,?,?,?,?,?)',(
                e['id'],
                e['trade_engine_id'],
                e['trade_engine_name'],
                e['trade_engine_title'],
                e['market_name'],
                e['market_title'],
                e['market_id'],
                e['marketplace']
             ))
        conn.commit()
    finally:
        conn.close()

In [119]:
def fill_boards(boards):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM boards')
        conn.commit()
        for e in boards:
            conn.execute('insert into boards values (?,?,?,?,?,?,?,?,?)',(
                e['id'],
                e['board_group_id'],
                e['engine_id'],
                e['market_id'],
                e['boardid'],
                e['board_title'],
                e['is_traded'],
                e['has_candles'],
                e['is_primary']
             ))
        conn.commit()
    finally:
        conn.close()

In [120]:
def fill_boardgroups(boardgroups):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM boardgroups')
        conn.commit()
        for e in boardgroups:
            conn.execute('insert into boardgroups values (?,?,?,?,?,?,?,?,?,?,?)',(
                e['id'],
                e['trade_engine_id'],
                e['trade_engine_name'],
                e['trade_engine_title'],
                e['market_id'],
                e['market_name'],
                e['name'],
                e['title'],
                e['is_default'],
                e['board_group_id'],
                e['is_traded']
             ))
        conn.commit()
    finally:
        conn.close()

In [121]:
def fill_durations(durations):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM durations')
        conn.commit()
        for e in durations:
            conn.execute('insert into durations values (?,?,?,?,?)',(
                e['interval'],
                e['duration'],
                e['days'],
                e['title'],
                e['hint']
             ))
        conn.commit()
    finally:
        conn.close()

In [122]:
def fill_securitytypes(securitytypes):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM securitytypes')
        conn.commit()
        for e in securitytypes:
            conn.execute('insert into securitytypes values (?,?,?,?,?,?)',(
                e['id'],
                e['trade_engine_id'],
                e['trade_engine_name'],
                e['trade_engine_title'],
                e['security_type_name'],
                e['security_type_title']
             ))
        conn.commit()
    finally:
        conn.close()

In [123]:
def fill_securitygroups(securitygroups):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM securitygroups')
        conn.commit()
        for e in securitygroups:
            conn.execute('insert into securitygroups values (?,?,?,?)',(
                e['id'],
                e['name'],
                e['title'],
                e['is_hidden']
             ))
        conn.commit()
    finally:
        conn.close()

In [124]:
def fill_securitycollections(securitycollections):
    conn = sqlite3.connect('moex.db')
    try:
        conn.execute('DELETE FROM securitycollections')
        conn.commit()
        for e in securitycollections:
            conn.execute('insert into securitycollections values (?,?,?,?)',(
                e['id'],
                e['name'],
                e['title'],
                e['security_group_id']
             ))
        conn.commit()
    finally:
        conn.close()

In [125]:
m = moex_markets()
print('engines ', m[1]['engines'][1][0])
fill_engines(m[1]['engines'][1])

print('markets ', m[1]['markets'][1][0])
fill_markets(m[1]['markets'][1])

print('boards ', m[1]['boards'][1][0])
fill_boards(m[1]['boards'][1])

print('boardgroups ', m[1]['boardgroups'][1][0])
fill_boardgroups(m[1]['boardgroups'][1])

print('durations ', m[1]['durations'][1][0])
fill_durations(m[1]['durations'][1])

print('securitytypes ', m[1]['securitytypes'][1][0])
fill_securitytypes(m[1]['securitytypes'][1])

print('securitygroups ', m[1]['securitygroups'][1][0])
fill_securitygroups(m[1]['securitygroups'][1])

print('securitycollections ', m[1]['securitycollections'][1][0])
fill_securitycollections(m[1]['securitycollections'][1])

url: http://iss.moex.com/iss/index.json?iss.json=extended
Cached http://iss.moex.com/iss/index.json?iss.json=extended
engines  {'id': 1, 'name': 'stock', 'title': 'Фондовый рынок и рынок депозитов'}
markets  {'id': 5, 'trade_engine_id': 1, 'trade_engine_name': 'stock', 'trade_engine_title': 'Фондовый рынок и рынок депозитов', 'market_name': 'index', 'market_title': 'Индексы фондового рынка', 'market_id': 5, 'marketplace': 'INDICES'}
boards  {'id': 260, 'board_group_id': 99, 'engine_id': 1, 'market_id': 1, 'boardid': 'EQTD', 'board_title': 'Т0 ETF (USD) - безадрес.', 'is_traded': 1, 'has_candles': 1, 'is_primary': 0}
boardgroups  {'id': 9, 'trade_engine_id': 1, 'trade_engine_name': 'stock', 'trade_engine_title': 'Фондовый рынок и рынок депозитов', 'market_id': 5, 'market_name': 'index', 'name': 'stock_index', 'title': 'Индексы', 'is_default': 1, 'board_group_id': 9, 'is_traded': 1}
durations  {'interval': 1, 'duration': 60, 'days': None, 'title': 'минута', 'hint': '1м'}
securitytypes  {

In [2]:
import requests
@cache
def get(url, params = None):
    return requests.get(url, params).json()

In [145]:
clear_cache()

In [74]:
# For testing
@cache
def do(url, params = {}):
    return params
print(do(url="123", params={"a": 1, "b": 2}))
print(do(url="456", params={"a": 1, "b": 2}))
print(do("123", {"a": 1, "b": 2}))
time.sleep(2)
print(do(url="123"))

url: 123?a=1&b=2
Cached 123?a=1&b=2
{'url': '123?a=1&b=2', 'timestamp': '2020-05-13T19:23:09.441742', 'result': {'a': 1, 'b': 2}}
url: 456?a=1&b=2
Cached 456?a=1&b=2
{'url': '456?a=1&b=2', 'timestamp': '2020-05-13T19:23:09.457036', 'result': {'a': 1, 'b': 2}}
url: 123?a=1&b=2
Cached 123?a=1&b=2
{'url': '123?a=1&b=2', 'timestamp': '2020-05-13T19:23:09.441742', 'result': {'a': 1, 'b': 2}}
url: 123?
Cached 123?
{'url': '123?', 'timestamp': '2020-05-13T19:24:18.867112', 'result': {}}


In [3]:
def moex_markets():
    return get('http://iss.moex.com/iss/index.json', {'iss.json': 'extended'})

In [164]:
def securities():
    start = 0
    limit = 100
    count = 1 # initial truish value
    results = []
    while count > 0:
        batch = get('http://iss.moex.com/iss/securities.json', {'iss.json': 'extended', 'limit': limit, 'start':start})
        results.append(batch)
        count = len(batch['securities']['data'])
        print ("From " + str(start) + "took " + str(count))
        start += count
    return results

In [166]:
x = securities()

url: http://iss.moex.com/iss/securities.json?limit=100&start=0
Cached http://iss.moex.com/iss/securities.json?limit=100&start=0
From 0took 100
url: http://iss.moex.com/iss/securities.json?limit=100&start=100
Cached http://iss.moex.com/iss/securities.json?limit=100&start=100
From 100took 100
url: http://iss.moex.com/iss/securities.json?limit=100&start=200
Cached http://iss.moex.com/iss/securities.json?limit=100&start=200
From 200took 100
url: http://iss.moex.com/iss/securities.json?limit=100&start=300
Cached http://iss.moex.com/iss/securities.json?limit=100&start=300
From 300took 100
url: http://iss.moex.com/iss/securities.json?limit=100&start=400
Cached http://iss.moex.com/iss/securities.json?limit=100&start=400
From 400took 100
url: http://iss.moex.com/iss/securities.json?limit=100&start=500
Cached http://iss.moex.com/iss/securities.json?limit=100&start=500
From 500took 100
url: http://iss.moex.com/iss/securities.json?limit=100&start=600
Cached http://iss.moex.com/iss/securities.json?

In [169]:
import json
print(json.dumps(moex_markets(), indent=2).encode('latin1').decode('unicode-escape'))

url: http://iss.moex.com/iss/index.json?iss.json=extended
Not cached http://iss.moex.com/iss/index.json?iss.json=extended
[
  {
    "charsetinfo": {
      "name": "utf-8"
    }
  },
  {
    "engines": [
      {
        "metadata": {
          "id": {
            "type": "int32"
          },
          "name": {
            "type": "string",
            "bytes": 45,
            "max_size": 0
          },
          "title": {
            "type": "string",
            "bytes": 765,
            "max_size": 0
          }
        }
      },
      [
        {
          "id": 1,
          "name": "stock",
          "title": "Фондовый рынок и рынок депозитов"
        },
        {
          "id": 2,
          "name": "state",
          "title": "Рынок ГЦБ (размещение)"
        },
        {
          "id": 3,
          "name": "currency",
          "title": "Валютный рынок"
        },
        {
          "id": 4,
          "name": "futures",
          "title": "Срочный рынок"
        },
        {


Cached http://iss.moex.com/iss/index.json


'http://iss.moex.com/iss/index.json'